In [3]:

import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import math
from scipy.stats import norm
from scipy.optimize import brentq
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from dotenv import load_dotenv


RISK_FREE_RATE = 0.03  




def bsm_put_price(S, K, r, sigma, T):
    """Black-Scholes European put price (no dividends)."""
    if T <= 0:
        return max(K - S, 0.0)
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma * sigma) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT

    call = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    put = call - S + K * math.exp(-r * T)
    return put


def bsm_vega(S, K, r, sigma, T):
    """Vega of option (derivative wrt volatility)."""
    if T <= 0:
        return 0.0
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma * sigma) * T) / (sigma * sqrtT)
    return S * norm.pdf(d1) * sqrtT


def implied_vol_put_nr_bisect(market_price, S, K, r, T,
                              sigma0=0.25, tol=1e-6, max_iter=100):

    if market_price is None or market_price <= 0 or T <= 0:
        return np.nan

    intrinsic = max(K - S, 0.0)
    if market_price < intrinsic:
        market_price = intrinsic

    # Newton-Raphson
    sigma = float(sigma0)
    for _ in range(max_iter):
        price = bsm_put_price(S, K, r, sigma, T)
        diff = price - market_price

        if abs(diff) < tol:
            return max(sigma, 0.0)

        vega = bsm_vega(S, K, r, sigma, T)
        if vega < 1e-8:
            break

        sigma = sigma - diff / vega
        sigma = min(max(sigma, 1e-6), 5.0)

    # brentq fallback
    def f(s):
        return bsm_put_price(S, K, r, s, T) - market_price

    low, high = 1e-6, 5.0
    try:
        if f(low) * f(high) > 0:
            high = 10.0
            if f(low) * f(high) > 0:
                return np.nan
        return brentq(f, low, high, xtol=tol, maxiter=200)
    except Exception:
        return np.nan




def get_otm_put_iv(ticker_symbol, r=RISK_FREE_RATE):
    ticker = yf.Ticker(ticker_symbol)
    ticker._download_options()

    today = dt.datetime.today().date()

    # convert expirations
    expirations = [
        dt.datetime.strptime(d, "%Y-%m-%d").date()
        for d in ticker._expirations
    ]

    # nearest expirations within 7 days. change as per your needs
    nearby = [d for d in expirations if 0 < (d - today).days < 7]
    if len(nearby) == 0:
        return None

    
    puts_list = []
    for exp in nearby:
        oc = ticker.option_chain(exp.strftime("%Y-%m-%d"))
        df_puts = oc.puts.copy()
        df_puts["expiration"] = exp
        puts_list.append(df_puts)

    all_puts = pd.concat(puts_list, ignore_index=True)

    
    spot = ticker.history(period="1d")["Close"].iloc[-1]

    otm_puts = all_puts[all_puts["strike"] < spot]
    if otm_puts.empty:
        return None

    
    otm_puts = otm_puts.sort_values(
        by=["openInterest", "volume"],
        ascending=False
    )
    row = otm_puts.iloc[0]


    bid = row.get("bid", np.nan)
    ask = row.get("ask", np.nan)
    last = row.get("lastPrice", np.nan)

    if (not np.isnan(bid)) and (not np.isnan(ask)) and ask > 0:
        mid = 0.5 * (bid + ask)
    elif (not np.isnan(last)) and last > 0:
        mid = last
    else:
        return None

    K = float(row["strike"])
    days = (row["expiration"] - today).days
    T = max(days / 252.0, 1e-6)

    iv = implied_vol_put_nr_bisect(
        market_price=mid,
        S=spot,
        K=K,
        r=r,
        T=T,
        sigma0=0.25
    )

    return {
        "ticker": ticker_symbol,
        "spot": spot,
        "strike": K,
        "expiry": row["expiration"],
        "iv": iv,
        "mid": mid,
        "volume": row.get("volume", np.nan),
        "openInterest": row.get("openInterest", np.nan),
    }


In [4]:


def realized_vol_21d(ticker_symbol):
    """
    Compute 21-day annualized realized volatility.
    """
    df = yf.download(ticker_symbol, period="2mo", interval="1d", progress=False)
    
    if df.shape[0] < 22:
        return np.nan

    df["ret"] = df["Close"].pct_change()
    rv = df["ret"].std() * np.sqrt(252)   
    return rv


stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA",
          "NVDA", "META", "JPM", "UNH", "V",
          "DIS", "MA", "NFLX", "PG", "BAC",
          "KO", "PFE", "WMT", "XOM", "HD"]


data = []
for s in stocks:
    iv_dict = get_otm_put_iv(s)
    if iv_dict is not None:
        data.append(iv_dict)

df_iv = pd.DataFrame(data)


df_iv["rv_21d"] = df_iv["ticker"].apply(realized_vol_21d)

# Compute IV / RV ratio
df_iv["iv_rv_ratio"] = df_iv["iv"] / df_iv["rv_21d"]

print(df_iv)


C:\Users\Dev Vatsayan\AppData\Local\Temp\ipykernel_20320\2979982983.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, period="2mo", interval="1d", progress=False)
C:\Users\Dev Vatsayan\AppData\Local\Temp\ipykernel_20320\2979982983.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, period="2mo", interval="1d", progress=False)
C:\Users\Dev Vatsayan\AppData\Local\Temp\ipykernel_20320\2979982983.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, period="2mo", interval="1d", progress=False)
C:\Users\Dev Vatsayan\AppData\Local\Temp\ipykernel_20320\2979982983.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker_symbol, period="2mo", interval="1d", progress=False)
C:\Users\Dev Vatsayan\AppData\Local\Temp\ipykernel_20320\2979982983.py:5: Fu

   ticker        spot  strike      expiry        iv   mid   volume  \
0    AAPL  283.100006   280.0  2025-12-05  0.207834  1.26  25900.0   
1    MSFT  486.739990   480.0  2025-12-05  0.224522  2.05   3614.0   
2   GOOGL  314.890015   310.0  2025-12-05  0.341902  2.57  10776.0   
3    AMZN  233.880005   230.0  2025-12-05  0.377245  2.15  11090.0   
4    TSLA  430.140015   425.0  2025-12-05  0.439363  5.80  45481.0   
5    NVDA  179.919998   175.0  2025-12-05  0.436645  1.45  99047.0   
6    META  640.869995   640.0  2025-12-05  0.276877  7.18   6540.0   
7     JPM  308.920013   300.0  2025-12-05  0.256157  0.63    571.0   
8     UNH  323.209991   300.0  2025-12-05  0.343698  0.10   1307.0   
9       V  330.390015   315.0  2025-12-05  0.265868  0.19    415.0   
10    DIS  106.769997    99.0  2025-12-05  0.322648  0.02   1417.0   
11     MA  543.969971   535.0  2025-12-05  0.224020  1.90    849.0   
12   NFLX  109.129997   108.0  2025-12-05  0.309132  0.95   3913.0   
13     PG  147.44000

In [5]:
df_iv

,ticker,spot,strike,expiry,iv,mid,volume,openInterest,rv_21d,iv_rv_ratio
0,AAPL,283.100006,280.0,2025-12-05,0.207834,1.26,25900.0,0,0.200388,1.037154
1,MSFT,486.739990,480.0,2025-12-05,0.224522,2.05,3614.0,0,0.199806,1.123700
2,GOOGL,314.890015,310.0,2025-12-05,0.341902,2.57,10776.0,0,0.334823,1.021142
3,AMZN,233.880005,230.0,2025-12-05,0.377245,2.15,11090.0,0,0.391015,0.964784
4,TSLA,430.140015,425.0,2025-12-05,0.439363,5.80,45481.0,0,0.502363,0.874593
5,NVDA,179.919998,175.0,2025-12-05,0.436645,1.45,99047.0,0,0.404610,1.079176
6,META,640.869995,640.0,2025-12-05,0.276877,7.18,6540.0,0,0.383308,0.722337
7,JPM,308.920013,300.0,2025-12-05,0.256157,0.63,571.0,0,0.211019,1.213906
8,UNH,323.209991,300.0,2025-12-05,0.343698,0.10,1307.0,0,0.273492,1.256703
9,V,330.390015,315.0,2025-12-05,0.265868,0.19,415.0,0,0.170165,1.562412


In [6]:

df_iv["iv_percentile"] = df_iv["iv"].rank(pct=True)


iv_rv_min = df_iv["iv_rv_ratio"].min()
iv_rv_max = df_iv["iv_rv_ratio"].max()
df_iv["iv_rv_scaled"] = (df_iv["iv_rv_ratio"] - iv_rv_min) / (iv_rv_max - iv_rv_min)

print(df_iv)


   ticker        spot  strike      expiry        iv   mid   volume  \
0    AAPL  283.100006   280.0  2025-12-05  0.207834  1.26  25900.0   
1    MSFT  486.739990   480.0  2025-12-05  0.224522  2.05   3614.0   
2   GOOGL  314.890015   310.0  2025-12-05  0.341902  2.57  10776.0   
3    AMZN  233.880005   230.0  2025-12-05  0.377245  2.15  11090.0   
4    TSLA  430.140015   425.0  2025-12-05  0.439363  5.80  45481.0   
5    NVDA  179.919998   175.0  2025-12-05  0.436645  1.45  99047.0   
6    META  640.869995   640.0  2025-12-05  0.276877  7.18   6540.0   
7     JPM  308.920013   300.0  2025-12-05  0.256157  0.63    571.0   
8     UNH  323.209991   300.0  2025-12-05  0.343698  0.10   1307.0   
9       V  330.390015   315.0  2025-12-05  0.265868  0.19    415.0   
10    DIS  106.769997    99.0  2025-12-05  0.322648  0.02   1417.0   
11     MA  543.969971   535.0  2025-12-05  0.224020  1.90    849.0   
12   NFLX  109.129997   108.0  2025-12-05  0.309132  0.95   3913.0   
13     PG  147.44000

In [7]:
#Compute Put-Call Skew using calculated call IV

def bsm_call_price(S, K, r, sigma, T):
    """Black-Scholes European call price (no dividends)."""
    if T <= 0:
        return max(S - K, 0.0)
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma * sigma) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

def implied_vol_call_nr_bisect(market_price, S, K, r, T,
                               sigma0=0.25, tol=1e-6, max_iter=100):
    """Compute call IV using Newton-Raphson + brentq fallback."""
    if market_price is None or market_price <= 0 or T <= 0:
        return np.nan

    intrinsic = max(S - K, 0.0)
    if market_price < intrinsic:
        market_price = intrinsic

    # Newton-Raphson
    sigma = float(sigma0)
    for _ in range(max_iter):
        price = bsm_call_price(S, K, r, sigma, T)
        diff = price - market_price
        vega = bsm_vega(S, K, r, sigma, T)
        if abs(diff) < tol:
            return max(sigma, 0.0)
        if vega < 1e-8:
            break
        sigma = min(max(sigma - diff / vega, 1e-6), 5.0)

    # Brentq fallback
    def f(s): return bsm_call_price(S, K, r, s, T) - market_price
    low, high = 1e-6, 5.0
    try:
        if f(low) * f(high) > 0:
            high = 10.0
            if f(low) * f(high) > 0:
                return np.nan
        return brentq(f, low, high, xtol=tol, maxiter=200)
    except Exception:
        return np.nan

def get_otm_call_iv(ticker_symbol, r=RISK_FREE_RATE):
    """Fetch highest liquidity OTM call and compute IV."""
    ticker = yf.Ticker(ticker_symbol)
    ticker._download_options()
    today = dt.datetime.today().date()
    expirations = [dt.datetime.strptime(d, "%Y-%m-%d").date() for d in ticker._expirations]
    nearby = [d for d in expirations if 0 < (d - today).days < 7]
    if len(nearby) == 0:
        return None

    calls_list = []
    for exp in nearby:
        oc = ticker.option_chain(exp.strftime("%Y-%m-%d"))
        df_calls = oc.calls.copy()
        df_calls["expiration"] = exp
        calls_list.append(df_calls)
    all_calls = pd.concat(calls_list, ignore_index=True)
    spot = ticker.history(period="1d")["Close"].iloc[-1]
    otm_calls = all_calls[all_calls["strike"] > spot]
    if otm_calls.empty:
        return None

    otm_calls = otm_calls.sort_values(by=["openInterest", "volume"], ascending=False)
    row = otm_calls.iloc[0]

    bid = row.get("bid", np.nan)
    ask = row.get("ask", np.nan)
    last = row.get("lastPrice", np.nan)

    if (not np.isnan(bid)) and (not np.isnan(ask)) and ask > 0:
        mid = 0.5 * (bid + ask)
    elif (not np.isnan(last)) and last > 0:
        mid = last
    else:
        return None

    K = float(row["strike"])
    days = (row["expiration"] - today).days
    T = max(days / 252.0, 1e-6)

    iv = implied_vol_call_nr_bisect(market_price=mid, S=spot, K=K, r=r, T=T)
    return iv

# Compute skew using calculated put & call IV
put_call_skew = []
for s in stocks:
    put_iv = df_iv.loc[df_iv["ticker"] == s, "iv"].values
    call_iv = get_otm_call_iv(s)
    if len(put_iv) == 0 or call_iv is None or np.isnan(call_iv) or call_iv == 0:
        put_call_skew.append(np.nan)
    else:
        put_call_skew.append(put_iv[0] / call_iv)

df_iv["put_call_skew"] = put_call_skew

print(df_iv)


   ticker        spot  strike      expiry        iv   mid   volume  \
0    AAPL  283.100006   280.0  2025-12-05  0.207834  1.26  25900.0   
1    MSFT  486.739990   480.0  2025-12-05  0.224522  2.05   3614.0   
2   GOOGL  314.890015   310.0  2025-12-05  0.341902  2.57  10776.0   
3    AMZN  233.880005   230.0  2025-12-05  0.377245  2.15  11090.0   
4    TSLA  430.140015   425.0  2025-12-05  0.439363  5.80  45481.0   
5    NVDA  179.919998   175.0  2025-12-05  0.436645  1.45  99047.0   
6    META  640.869995   640.0  2025-12-05  0.276877  7.18   6540.0   
7     JPM  308.920013   300.0  2025-12-05  0.256157  0.63    571.0   
8     UNH  323.209991   300.0  2025-12-05  0.343698  0.10   1307.0   
9       V  330.390015   315.0  2025-12-05  0.265868  0.19    415.0   
10    DIS  106.769997    99.0  2025-12-05  0.322648  0.02   1417.0   
11     MA  543.969971   535.0  2025-12-05  0.224020  1.90    849.0   
12   NFLX  109.129997   108.0  2025-12-05  0.309132  0.95   3913.0   
13     PG  147.44000

In [8]:

# Weight parameters (just for demonstration, adjust as needed)
weight_iv_percentile = 0.4
weight_iv_rv_scaled = 0.4
weight_put_call_skew = 0.2

# Scale put-call skew to 0-1 for consistency
skew_min = np.nanmin(df_iv["put_call_skew"])
skew_max = np.nanmax(df_iv["put_call_skew"])
df_iv["skew_scaled"] = (df_iv["put_call_skew"] - skew_min) / (skew_max - skew_min)

# Compute final score
df_iv["score"] = (
    weight_iv_percentile * df_iv["iv_percentile"] +
    weight_iv_rv_scaled * df_iv["iv_rv_scaled"] +
    weight_put_call_skew * df_iv["skew_scaled"]
)

# Generate binary signal based on threshold
threshold = 0.75
df_iv["signal"] = df_iv["score"].apply(lambda x: 1 if x >= threshold else 0)

print(df_iv[["ticker", "score", "signal"]])


   ticker     score  signal
0    AAPL  0.280012       0
1    MSFT  0.410949       0
2   GOOGL  0.507535       0
3    AMZN  0.539269       0
4    TSLA  0.529155       0
5    NVDA  0.622020       0
6    META  0.294888       0
7     JPM  0.545385       0
8     UNH  0.675578       0
9       V  0.779850       1
10    DIS  0.694911       0
11     MA  0.435906       0
12   NFLX       NaN       0
13     PG  0.470872       0
14    BAC  0.573384       0
15     KO  0.323479       0
16    PFE  0.113604       0
17    WMT  0.067339       0
18    XOM  0.421763       0
19     HD  0.655227       0


In [9]:
# Separate stocks with indicator = 1
df_flagged = df_iv[df_iv["signal"] == 1].copy()


df_flagged.reset_index(drop=True, inplace=True)

print("Stocks with signal = 1:")
print(df_flagged)


Stocks with signal = 1:
  ticker        spot  strike      expiry        iv   mid  volume  \
0      V  330.390015   315.0  2025-12-05  0.265868  0.19   415.0   

   openInterest    rv_21d  iv_rv_ratio  iv_percentile  iv_rv_scaled  \
0             0  0.170165     1.562412            0.5           1.0   

   put_call_skew  skew_scaled    score  signal  
0       1.334295     0.899248  0.77985       1  


In [ ]:
#load_dotenv(r"C:your_path/iv.env")

In [11]:
#smtp_server = "smtp.gmail.com"
#smtp_port = 587
#email_id = os.getenv("email_id")
#email_password = os.getenv("email_password")

#(Use a dotenv file to store sensitive information)

In [12]:
#from tabulate import tabulate



In [13]:
#table_str = tabulate(df_flagged, headers="keys", tablefmt="pretty", showindex=False)

In [ ]:
#def send_email(subject, body, recipient_email):
    #try:
        #msg = MIMEMultipart()
        #msg['From'] = email_id
        #msg['To'] = recipient_email
        #msg['Subject'] = subject

    # Attach the body text
        #msg.attach(MIMEText(body, 'plain'))

    # Send the email
        #with smtplib.SMTP(smtp_server, smtp_port) as server:
          #server.starttls()
          #server.login(email_id, email_password)
          #server.sendmail(email_id, recipient_email, msg.as_string())

        #print("Email sent successfully")
    #except Exception as e:  
        #print(f"Failed to send email: {e}")

#if __name__ == "__main__":
    #subject = "Flagged Stocks with Signal = 1"
    #body = (
    #"Hello,\n\n"
    #"Here is your daily alert for stocks that triggered **Signal = 1**.\n"
    #"These stocks may be showing high risk based on IV score.\n\n"
    #"-----------------------------------------\n"
    #"SUMMARY\n"
    #f"Total flagged stocks: {len(df_flagged)}\n"
    #"-----------------------------------------\n\n"
    #"📊 FLAGGED STOCK DETAILS:\n"
    #f"{table_str}\n\n"
    #"-----------------------------------------\n"
    #"This is an automated alert. Monitor these tickers closely.\n"
    #"Regards,\n"
    #"Your bot")

    #recipient_email = "putyourmail"



In [15]:
#send_email(subject, body, recipient_email)